In [1]:
import torch
import torch.nn as nn


class DeepGRUModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(DeepGRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)  
        self.gru = nn.GRU(embedding_dim, 128, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
        self.fc1 = nn.Linear(128 * 2, 64) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)  

        
    def forward(self, x):
        x = self.embedding(x)  
        x, _ = self.gru(x)
        x = torch.mean(x, dim=1)  
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x) 
        return x  

In [2]:
model = DeepGRUModel(195, 128)

In [3]:
model.load_state_dict(torch.load("model.pth", weights_only=False))
model.eval()



DeepGRUModel(
  (embedding): Embedding(195, 128, padding_idx=0)
  (gru): GRU(128, 128, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [4]:
import pandas as pd


df = pd.read_csv("password_dataset.csv")

passwords = df["password"].astype(str).tolist()
y = df["is_meaningful"].values

unique_chars = sorted(set("".join(passwords)))
char_to_idx = {char: i + 1 for i, char in enumerate(unique_chars)}

max_len = max(len(p) for p in passwords)


def predict_password_strength(password, model, device="cpu"):
    encoded_pw = encode_password(password)
    
    input_tensor = encoded_pw.unsqueeze(0).to(device) 
    model.eval() 
    with torch.no_grad():
        output = model(input_tensor).squeeze()  
        prob = torch.sigmoid(output)  

    return prob.item()  


def encode_password(password):
    encoded = [char_to_idx.get(char, 0) for char in password]
    padded = encoded + [0] * (max_len - len(encoded))
    return torch.tensor(padded[:max_len], dtype=torch.long)



In [5]:

passwords = ["12345", "testpassword", "mysecret", "!=)21mkı213j=%", "lolitsmeaningful", "pojfkappascore12312_?_", "293ıp***1?240ıdaks?'test", "güvenlişifre123"]

for password in passwords:
    probability = predict_password_strength(password, model)
    print(f"Password '{password}' meaningful probability: {probability:.4f}")


Password '12345' meaningful probability: 1.0000
Password 'testpassword' meaningful probability: 1.0000
Password 'mysecret' meaningful probability: 1.0000
Password '!=)21mkı213j=%' meaningful probability: 0.2387
Password 'lolitsmeaningful' meaningful probability: 1.0000
Password 'pojfkappascore12312_?_' meaningful probability: 1.0000
Password '293ıp***1?240ıdaks?'test' meaningful probability: 0.9958
Password 'güvenlişifre123' meaningful probability: 1.0000
